## Library

In [ ]:
!pip install bpemb
!pip install gensim
!python -m spacy download en_core_web_sm
!pip install mpld3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 41.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import torch
import collections
import random
import numpy as np
from torch import nn
from bpemb import BPEmb
import pandas as pd
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForMaskedLM
from transformers import AdamW
from sklearn.model_selection import train_test_split
from torch.cuda.amp import GradScaler, autocast

## Dataset Loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = '/content/drive/MyDrive/NLP/tydi_xor_re.xlsx'
df = pd.read_excel(file_path)

In [ ]:
df.head()

,question,context,lang,answerable,answer_start,answer,answer_inlang
0,উইকিলিকস কত সালে সর্বপ্রথম ইন্টারনেটে প্রথম তথ...,WikiLeaks () is an international non-profit or...,bn,True,182,2006,NaN
1,দ্বিতীয় বিশ্বযুদ্ধে কোন দেশ পরাজিত হয় ?,The war in Europe concluded with an invasion o...,bn,True,48,Germany,NaN
2,মার্কিন যুক্তরাষ্ট্রের সংবিধান অনুযায়ী মার্কিন...,Same-sex marriage in the United States expande...,bn,False,-1,no,NaN
3,আরব-ইসরায়েলি যুদ্ধে আরবের মোট কয়জন সৈন্যের মৃ...,The exact number of Arab casualties is unknown...,bn,True,39,unknown,NaN
4,বিশ্বে প্রথম পুঁজিবাদী সমাজ কবে গড়ে ওঠে ?,"As Thomas Hall (2000) notes, ""The Sung Empire ...",bn,True,1219,17th century,NaN


# Language_Models_For_question

## Tokezination_Fi_Ru_Ja

In [ ]:
# Load the multilingual BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

def tokenize_question(row):
    if row['lang'] in ['fi', 'ru', 'ja']:
        return tokenizer.tokenize(row['question'])
    else:
        return None

df['tokens'] = df.apply(tokenize_question, axis=1)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### tokenizer display

In [ ]:
df.head(3000)

,question,context,lang,answerable,answer_start,answer,answer_inlang
0,উইকিলিকস কত সালে সর্বপ্রথম ইন্টারনেটে প্রথম তথ...,WikiLeaks () is an international non-profit or...,bn,True,182,2006,NaN
1,দ্বিতীয় বিশ্বযুদ্ধে কোন দেশ পরাজিত হয় ?,The war in Europe concluded with an invasion o...,bn,True,48,Germany,NaN
2,মার্কিন যুক্তরাষ্ট্রের সংবিধান অনুযায়ী মার্কিন...,Same-sex marriage in the United States expande...,bn,False,-1,no,NaN
3,আরব-ইসরায়েলি যুদ্ধে আরবের মোট কয়জন সৈন্যের মৃ...,The exact number of Arab casualties is unknown...,bn,True,39,unknown,NaN
4,বিশ্বে প্রথম পুঁজিবাদী সমাজ কবে গড়ে ওঠে ?,"As Thomas Hall (2000) notes, ""The Sung Empire ...",bn,True,1219,17th century,NaN
...,...,...,...,...,...,...,...
2995,オンラインゲームの起源はいつ？,The history of online games dates back to the ...,ja,True,200,1978,NaN
2996,唐は中国統一した？,The Tang dynasty (; ) or the Tang Empire was a...,ja,False,-1,no,NaN
2997,ルッキズムにより訴訟に発展した事例はある？,Lookism is discriminatory treatment toward peo...,ja,False,-1,no,NaN
2998,カーダール・ヤーノシュは虐殺を実行したことがある？,"In 1962, six years after the Hungarian Revolut...",ja,False,-1,no,NaN


## Dataset of Finnish language

In [ ]:
# 1. Filter the DataFrame for Finnish language questions
fi_df = df[df['lang'] == 'fi']

# 2. Split the DataFrame into training and validation sets
fi_train_df, fi_val_df = train_test_split(fi_df, test_size=0.2, random_state=42)

## Dataset for Russian language

In [ ]:
# 1. Filter the DataFrame for Russian language questions
ru_df = df[df['lang'] == 'ru']

# 2. Split the DataFrame into training and validation sets
ru_train_df, ru_val_df = train_test_split(ru_df, test_size=0.2, random_state=42)

## Dataset for Japanese Language

In [ ]:
# 1. Filter the DataFrame for Japanese language questions
ja_df = df[df['lang'] == 'ja']

# 2. Split the DataFrame into training and validation sets
ja_train_df, ja_val_df = train_test_split(ja_df, test_size=0.2, random_state=42)

## Dataset Loader

In [ ]:
class QuestionDataset(Dataset):
    def __init__(self, questions, tokens, tokenizer, max_length=128):
        self.questions = questions
        self.tokens = tokens
        self.tokenizer = tokenizer  # Store the tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        # Convert tokens to input IDs
        input_ids = self.tokenizer.convert_tokens_to_ids(self.tokens[idx])
        attention_mask = [1] * len(input_ids)

        # Mask a random token for language modeling
        mask_idx = None
        if len(input_ids) > 1:
            mask_idx = torch.randint(0, len(input_ids), (1,)).item()
            input_ids[mask_idx] = self.tokenizer.mask_token_id

        # Padding to max length
        padding_length = self.max_length - len(input_ids)
        if padding_length > 0:
            input_ids += [0] * padding_length
            attention_mask += [0] * padding_length

        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'masked_index': mask_idx
        }


## BERT_Model_For_Japanese_Language

In [ ]:
# Create dataset and dataloaders for training and validation
train_dataset_ja = QuestionDataset(ja_train_df['question'].tolist(), ja_train_df['tokens'].tolist(), tokenizer)
val_dataset_ja = QuestionDataset(ja_val_df['question'].tolist(), ja_val_df['tokens'].tolist(), tokenizer)
train_dataloader_ja = DataLoader(train_dataset_ja, batch_size=8, shuffle=True)
val_dataloader_ja = DataLoader(val_dataset_ja, batch_size=8, shuffle=False)

model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')
model.train()
model.to('cuda')  # Moving model to GPU

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)


# Training Loop
for epoch in range(3):
    for batch in train_dataloader_ja:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)

        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

# 4. Validate the Model
model.eval()

# Function to get probabilities on the validation set
def validate_model(data_loader):
    total_loss = 0
    total_correct = 0
    total_tokens = 0  # To count the total number of tokens for accuracy calculation

    model.eval()
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to('cuda')
            attention_mask = batch['attention_mask'].to('cuda')

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            total_loss += outputs.loss.item()

            # predicted token IDs
            predictions = outputs.logits.argmax(dim=-1)

            # Compare predictions with the input_ids
            for i in range(predictions.size(0)):
                for j in range(predictions.size(1)):
                    if input_ids[i][j] != 0:
                        total_tokens += 1
                        if predictions[i][j] == input_ids[i][j]:
                            total_correct += 1

    avg_loss = total_loss / len(data_loader)
    accuracy = total_correct / total_tokens if total_tokens > 0 else 0  # Calculate accuracy

    print(f"Validation Loss: {avg_loss:.4f}, Validation Accuracy: {accuracy:.4f}")


# Validate the model
validate_model(val_dataloader_ja)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Validation Loss: 0.0276, Validation Accuracy: 0.9558


## BERT_Model_For_Russian_Language

In [ ]:
# Create dataset and dataloaders for training and validation
train_dataset_ru = QuestionDataset(ru_train_df['question'].tolist(), ru_train_df['tokens'].tolist(), tokenizer)
val_dataset_ru = QuestionDataset(ru_val_df['question'].tolist(), ru_val_df['tokens'].tolist(), tokenizer)
train_dataloader_ru = DataLoader(train_dataset_ru, batch_size=8, shuffle=True)
val_dataloader_ru = DataLoader(val_dataset_ru, batch_size=8, shuffle=False)

# 3. Load the model and set it to training mode
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')
model.train()
model.to('cuda')  # Moving model to GPU

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training Loop
for epoch in range(3):
    for batch in train_dataloader_ru:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)

        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

# 4. Validate the Model
model.eval()

# Function to get probabilities on the validation set
def validate_model(data_loader):
    total_loss = 0
    total_correct = 0
    total_tokens = 0  # To count the total number of tokens for accuracy calculation

    model.eval()
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to('cuda')
            attention_mask = batch['attention_mask'].to('cuda')

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            total_loss += outputs.loss.item()

            # predicted token IDs
            predictions = outputs.logits.argmax(dim=-1)

            # Compare predictions with the input_ids
            for i in range(predictions.size(0)):
                for j in range(predictions.size(1)):
                    if input_ids[i][j] != 0:
                        total_tokens += 1
                        if predictions[i][j] == input_ids[i][j]:
                            total_correct += 1

    avg_loss = total_loss / len(data_loader)
    accuracy = total_correct / total_tokens if total_tokens > 0 else 0  # Calculate accuracy

    print(f"Validation Loss: {avg_loss:.4f}, Validation Accuracy: {accuracy:.4f}")

# Validate the model
validate_model(val_dataloader_ru)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Validation Loss: 0.0017, Validation Accuracy: 0.9971


## BERT_Model_For_Finnish_Language

In [ ]:
# Create dataset and dataloaders for training and validation
train_dataset_fi = QuestionDataset(fi_train_df['question'].tolist(), fi_train_df['tokens'].tolist(), tokenizer)
val_dataset_fi = QuestionDataset(fi_val_df['question'].tolist(), fi_val_df['tokens'].tolist(), tokenizer)
train_dataloader_fi = DataLoader(train_dataset_fi, batch_size=8, shuffle=True)
val_dataloader_fi = DataLoader(val_dataset_fi, batch_size=8, shuffle=False)

model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')
model.train()
model.to('cuda')  # Moving model to GPU

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training Loop
for epoch in range(3):
    for batch in train_dataloader_fi:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)

        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

# 4. Validate the Model
model.eval()

# Function to get probabilities on the validation set
def validate_model(data_loader):
    total_loss = 0
    total_correct = 0
    total_tokens = 0  # To count the total number of tokens for accuracy calculation

    model.eval()
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to('cuda')
            attention_mask = batch['attention_mask'].to('cuda')

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            total_loss += outputs.loss.item()

            # predicted token IDs
            predictions = outputs.logits.argmax(dim=-1)

            # Compare predictions with the input_ids
            for i in range(predictions.size(0)):
                for j in range(predictions.size(1)):
                    if input_ids[i][j] != 0:
                        total_tokens += 1
                        if predictions[i][j] == input_ids[i][j]:
                            total_correct += 1

    avg_loss = total_loss / len(data_loader)
    accuracy = total_correct / total_tokens if total_tokens > 0 else 0  # Calculate accuracy

    print(f"Validation Loss: {avg_loss:.4f}, Validation Accuracy: {accuracy:.4f}")

# Validate the model
validate_model(val_dataloader_fi)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Validation Loss: 0.0047, Validation Accuracy: 0.9889


# Language_Model_For_context

## Context_Tokezination_Fi_Ru_Ja

In [ ]:
# Load the multilingual BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

def tokenize_context(row):
    if row['lang'] in ['fi', 'ru', 'ja']:
        return tokenizer.tokenize(row['context'])
    else:
        return None

df['context_tokens'] = df.apply(tokenize_context, axis=1)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### tokenizer display

In [ ]:
df.head(3000)

,question,context,lang,answerable,answer_start,answer,answer_inlang,context_tokens
0,উইকিলিকস কত সালে সর্বপ্রথম ইন্টারনেটে প্রথম তথ...,WikiLeaks () is an international non-profit or...,bn,True,182,2006,NaN,None
1,দ্বিতীয় বিশ্বযুদ্ধে কোন দেশ পরাজিত হয় ?,The war in Europe concluded with an invasion o...,bn,True,48,Germany,NaN,None
2,মার্কিন যুক্তরাষ্ট্রের সংবিধান অনুযায়ী মার্কিন...,Same-sex marriage in the United States expande...,bn,False,-1,no,NaN,None
3,আরব-ইসরায়েলি যুদ্ধে আরবের মোট কয়জন সৈন্যের মৃ...,The exact number of Arab casualties is unknown...,bn,True,39,unknown,NaN,None
4,বিশ্বে প্রথম পুঁজিবাদী সমাজ কবে গড়ে ওঠে ?,"As Thomas Hall (2000) notes, ""The Sung Empire ...",bn,True,1219,17th century,NaN,None
...,...,...,...,...,...,...,...,...
2995,オンラインゲームの起源はいつ？,The history of online games dates back to the ...,ja,True,200,1978,NaN,"[The, history, of, online, games, dates, back,..."
2996,唐は中国統一した？,The Tang dynasty (; ) or the Tang Empire was a...,ja,False,-1,no,NaN,"[The, Tang, dynasty, (, ;, ), or, the, Tang, E..."
2997,ルッキズムにより訴訟に発展した事例はある？,Lookism is discriminatory treatment toward peo...,ja,False,-1,no,NaN,"[Look, ##ism, is, disc, ##rimi, ##nato, ##ry, ..."
2998,カーダール・ヤーノシュは虐殺を実行したことがある？,"In 1962, six years after the Hungarian Revolut...",ja,False,-1,no,NaN,"[In, 1962, ,, six, years, after, the, Hungaria..."


## Spliting_dataset_for_context

In [ ]:
context_train_df, context_val_df = train_test_split(df[df['lang'].isin(['fi', 'ru', 'ja'])], test_size=0.2, random_state=42)

In [ ]:
context_train_df.head()

,question,context,lang,answerable,answer_start,answer,answer_inlang,context_tokens
7290,Османская империя воевала на стороне Германско...,"In August 1944, the Soviet Army entered Bulgar...",ru,False,-1,no,NaN,"[In, August, 1944, ,, the, Soviet, Army, enter..."
7265,Почему распался Демократический союз?,By June 2015 tensions within the two parties f...,ru,True,139,differences between the positions of the UDC l...,NaN,"[By, June, 2015, tensions, within, the, two, p..."
4327,保守統一党が結成されたのはいつ,"The Justice Unity Party (, ) was a Thai pro-mi...",ja,True,132,1991,NaN,"[The, Justice, Unity, Party, (, ,, ), was, a, ..."
3445,イヴァン雷帝の通称はなぜつけられましたか？,Ivan the Terrible (born 1911) is the nickname ...,ja,True,407,extremely violent behavior,NaN,"[Ivan, the, Ter, ##rib, ##le, (, born, 1911, )..."
3987,ルネサンス期は西暦何年から何年まで？,"The Renaissance (, ) is a period in European h...",ja,True,84,14th and 17th centuries,NaN,"[The, Renaissance, (, ,, ), is, a, period, in,..."


## Dataset Loader

In [ ]:
# Create a Dataset class
class ContextDataset(Dataset):
    def __init__(self, context, context_tokens, tokenizer, max_length=128):
        self.context = context
        self.context_tokens = context_tokens
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.context_tokens)

    def __getitem__(self, idx):
      # Token IDs and attention masks
      input_ids = self.tokenizer.convert_tokens_to_ids(self.context_tokens[idx])

      # Truncate if the sequence is longer than max_length
      if len(input_ids) > self.max_length:
          input_ids = input_ids[:self.max_length]

      attention_mask = [1] * len(input_ids)

      # Mask a random token for language modeling (excluding special tokens like [CLS], [SEP], etc.)
      mask_idx = None
      if len(input_ids) > 1:
          mask_idx = torch.randint(0, len(input_ids), (1,)).item()
          input_ids[mask_idx] = self.tokenizer.mask_token_id

      # Padding to max length
      padding_length = self.max_length - len(input_ids)
      if padding_length > 0:
          input_ids += [0] * padding_length
          attention_mask += [0] * padding_length

      return {
          'input_ids': torch.tensor(input_ids, dtype=torch.long),
          'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
          'masked_index': mask_idx
      }


## BERT_Model_For_Context

In [ ]:
# Create dataset and dataloaders for training and validation
train_dataset_context = ContextDataset(context_train_df['context'].tolist(), context_train_df['context_tokens'].tolist(), tokenizer)
val_dataset_context = ContextDataset(context_val_df['context'].tolist(), context_val_df['context_tokens'].tolist(), tokenizer)
train_dataloader_context = DataLoader(train_dataset_context, batch_size=8, shuffle=True)
val_dataloader_context = DataLoader(val_dataset_context, batch_size=8, shuffle=False)

model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')
model.train()
model.to('cuda')  # Moving model to GPU

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training Loop
for epoch in range(3):
    for batch in train_dataloader_context:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)

        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

# 4. Validate the Model
model.eval()

# Function to get probabilities on the validation set
def validate_model(data_loader):
    total_loss = 0
    total_correct = 0
    total_tokens = 0  # To count the total number of tokens for accuracy calculation

    model.eval()
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to('cuda')
            attention_mask = batch['attention_mask'].to('cuda')

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            total_loss += outputs.loss.item()

            # predicted token IDs
            predictions = outputs.logits.argmax(dim=-1)

            # Compare predictions with the input_ids
            for i in range(predictions.size(0)):
                for j in range(predictions.size(1)):
                    if input_ids[i][j] != 0:
                        total_tokens += 1
                        if predictions[i][j] == input_ids[i][j]:
                            total_correct += 1

    avg_loss = total_loss / len(data_loader)
    accuracy = total_correct / total_tokens if total_tokens > 0 else 0  # Calculate accuracy

    print(f"Validation Loss: {avg_loss:.4f}, Validation Accuracy: {accuracy:.4f}")

# Validate the model
validate_model(val_dataloader_context)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Validation Loss: 0.0000, Validation Accuracy: 1.0000
